In [ ]:
import os
import pandas as pd

# Change to correct dataset directory
root_directory = '/Users/divya/Documents/Data_collection/final_correct_dataset_with_default//'

# Create an empty list to store DataFrames
dataframes = []

# Walk through the root directory and its subdirectories
for dirpath, dirnames, filenames in os.walk(root_directory):
    
    for filename in filenames:
        if filename.endswith('.csv'):
            file_path = os.path.join(dirpath, filename)
            df = pd.read_csv(file_path)
            dataframes.append(df)

# Concatenate all DataFrames into one big DataFrame
big_dataframe = pd.concat(dataframes, ignore_index=True)
big_dataframe

In [ ]:
# Normalisation
# Define min-max values for each column (modify these as needed)
column_min_max_values = {
    'angle': (big_dataframe['angle'].min(), big_dataframe['angle'].max()),
    'fsr1': (big_dataframe['fsr1'].min(), big_dataframe['fsr1'].max()),
    'fsr2': (big_dataframe['fsr2'].min(), big_dataframe['fsr2'].max()),
    'fsr3': (big_dataframe['fsr3'].min(), big_dataframe['fsr3'].max()),
    'fsr4': (big_dataframe['fsr4'].min(), big_dataframe['fsr4'].max()),
    'acc2x':(-15.0,15.0),
    'acc2y':(-15.0,15.0),
    'acc2z':(-15.0,15.0),
    'gyro1':(-6.0,6.0),
    'gyro2':(-6.0,6.0),
    'gyro3':(-6.0,6.0),
    #'orient1': (0, 350),
    #'orient2': (0, 100),
    #'orient3': (0, 100),
}

root_directory = '/Users/divya/Documents/Data_collection/final_correct_dataset_with_default//'

# Create an empty list to store DataFrames
dataframes = []

# Walk through the root directory and its subdirectories
for dirpath, dirnames, filenames in os.walk(root_directory):
    
    for filename in filenames:
        if filename.endswith('.csv'):
            file_path = os.path.join(dirpath, filename)
            df = pd.read_csv(file_path)
            normalized_data = df.copy()
            for column, (min_val, max_val) in column_min_max_values.items():
                normalized_data[column] = (df[column] - min_val) / (max_val - min_val)
            dataframes.append(normalized_data)

# Concatenate all DataFrames into one big DataFrame
big_dataframe = pd.concat(dataframes, ignore_index=True)
big_dataframe

In [ ]:
# This block is to select the 2 out of 4 FSRs for all the participants
# Group by 'participant'
grouped_df = big_dataframe.groupby('participant')

# Mapping of participant to the columns to copy
participant_column_mapping = {
    1: ['fsr2', 'fsr4'],
    2: ['fsr2', 'fsr4'],
    3: ['fsr2', 'fsr4'],
    4: ['fsr2', 'fsr4'],
    5: ['fsr2', 'fsr4'],
    6: ['fsr2', 'fsr4'],
    9: ['fsr2', 'fsr4'],
    10: ['fsr2', 'fsr3'],
    11: ['fsr2', 'fsr4'],
    12: ['fsr1', 'fsr3'],
    13: ['fsr2', 'fsr4'],
    14: ['fsr2', 'fsr3'],

}

# Create new columns 'A' and 'B' to store the merged values
big_dataframe['FSR1'] = ''
big_dataframe['FSR2'] = ''

# Copy columns within each group into new columns A and B
for participant_id, cols_to_copy in participant_column_mapping.items():
    mask = big_dataframe['participant'] == participant_id
    big_dataframe.loc[mask, 'FSR1'] += big_dataframe.loc[mask, cols_to_copy[0]].astype(str) + ','
    big_dataframe.loc[mask, 'FSR2'] += big_dataframe.loc[mask, cols_to_copy[1]].astype(str) + ','

# Remove the trailing commas
big_dataframe['FSR1'] = big_dataframe['FSR1'].str.rstrip(',')
big_dataframe['FSR2'] = big_dataframe['FSR2'].str.rstrip(',')

# Display the modified DataFrame
#print(big_dataframe1)
big_dataframe


In [ ]:
big_dataframe['FSR1'] = big_dataframe['FSR2'].astype(float)
big_dataframe['FSR2'] = big_dataframe['FSR2'].astype(float)

In [ ]:
big_dataframe=big_dataframe.drop(columns=['fsr1','fsr2','fsr3','fsr4'])

In [ ]:
train_participants = [1, 2, 3, 4, 5, 6, 9, 12, 13]
test_participants = [10, 11, 14]

# Filter train_set and test_set based on 'participant' column
train_set = big_dataframe[big_dataframe['participant'].isin(train_participants)].copy()
test_set = big_dataframe[big_dataframe['participant'].isin(test_participants)].copy()

In [ ]:
unique_serial_numbers = train_set['serialnumber'].unique()

X_train = []
y_train = []

# Iterate through unique serial numbers and append arrays and class values to the lists
for serial_number in unique_serial_numbers:
    subset_df = train_set[train_set['serialnumber'] == serial_number]
    acc2x_y_array = subset_df[['gyro1','gyro2','gyro3']].to_numpy()
    class_value = subset_df['class'].iloc[0]  # Get the first value of the 'class' column
    X_train.append(acc2x_y_array)
    y_train.append(class_value)


In [ ]:
unique_serial_numbers = test_set['serialnumber'].unique()

X_test = []
y_test = []

for serial_number in unique_serial_numbers:
    subset_df = test_set[test_set['serialnumber'] == serial_number]
    acc2x_y_array = subset_df[['gyro1','gyro2','gyro3']].to_numpy()
    class_value = subset_df['class'].iloc[0]  # Get the first value of the 'class' column
    X_test.append(acc2x_y_array)
    y_test.append(class_value)


In [ ]:
from tslearn.utils import to_time_series_dataset
X_train = to_time_series_dataset(X_train)
X_test = to_time_series_dataset(X_test)

In [ ]:
!pip install tslearn
!pip install scikit-learn

In [ ]:
from sklearn.model_selection import train_test_split

X = X_train
y = y_train

# Print the sizes of the resulting sets
print("X_train size:", len(X_train))
print("X_test size:", len(X_test))
print("y_train size:", len(y_train))
print("y_test size:", len(y_test))

In [ ]:
import numpy
from scipy.spatial.distance import cdist
import matplotlib.pyplot as plt
from tslearn.preprocessing import TimeSeriesScalerMeanVariance
from tslearn import metrics
import time
import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
import seaborn as sns

In [ ]:
#knn dtw
from tslearn.neighbors import KNeighborsTimeSeriesClassifier


In [ ]:
tsc_dtw = KNeighborsTimeSeriesClassifier(n_neighbors=2, metric="dtw", n_jobs = -1)

start_time = time.time()
tsc_dtw.fit(X_train, y_train)
training_time = time.time() - start_time
print("Training time:", training_time, "seconds")

In [ ]:
#cv
logo = LeaveOneGroupOut()
X_train_reshaped = np.array(X_train)
accuracy_scores = cross_val_score(tsc_dtw, X_train_reshaped, y_train, groups=participant, cv=logo)

average_accuracy = np.mean(accuracy_scores)

print("Accuracy scores for each fold:", accuracy_scores)
print("Average Accuracy:", average_accuracy)

In [ ]:
start_time_single_prediction = time.time()
y_dtw = tsc_dtw.predict(X_test)
single_prediction_time = time.time() - start_time_single_prediction
print("Single Prediction time:", single_prediction_time/len(y_test), "seconds")
 
print("Accuracy using Knn: {0:.2f}".format(accuracy_score(y_dtw,y_test)))

In [ ]:
class_report_svm = classification_report(y_test, y_dtw)
print("Classification Report for SVM:\n", class_report_svm)

In [ ]:
class_labels = ["Default","Stop", "Speed_Up", "Slow_Down", "Change_Direction", "Repeat", "Reset"]

label_mapping = {
    0: "Default",
    1: "Stop",
    2: "Speed_Up",
    3: "Slow_Down",
    4: "Change_Direction",
    5: "Repeat",
    6: "Reset"
}

y_test_mapped = [label_mapping[label] for label in y_test]
y_pred_mapped = [label_mapping[label] for label in y_dtw]

conf_matrix = confusion_matrix(y_test_mapped, y_pred_mapped, labels=class_labels)

confusion_df = pd.DataFrame(conf_matrix, index=class_labels, columns=class_labels)
confusion_df

In [ ]:
# Confusion matrix with percentage
conf_matrix = confusion_matrix(y_test_mapped, y_pred_mapped, labels=class_labels)

confusion_df = pd.DataFrame(conf_matrix, index=class_labels, columns=class_labels)
confusion_df_percent = confusion_df.divide(confusion_df.sum(axis=1), axis=0) * 100

# Generate annotations with both actual number and percentage
annotations = confusion_df.astype(str) + '\n' + confusion_df_percent.round(2).astype(str) + '%'

# Generate a heatmap with custom colors and annotation showing both number and percentage
plt.figure(figsize=(10, 8))
sns.heatmap(confusion_df, annot=annotations, fmt='', cmap="Blues", cbar_kws={'label': 'Counts'})

plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')

plt.show()

In [ ]:
#SVM
from tslearn.svm import TimeSeriesSVC
clf = TimeSeriesSVC(C=1.0, kernel="gak")

start_time = time.time()
clf.fit(X_train, y_train)

# Calculate the training time
training_time = time.time() - start_time
print("Training time:", training_time, "seconds")



In [ ]:
#cv-svm
logo = LeaveOneGroupOut()
clf = TimeSeriesSVC(C=1.0, kernel="gak")
X_train_reshaped = np.array(X_train)
accuracy_scores = cross_val_score(clf, X_train_reshaped, y_train, groups=participant, cv=logo)

average_accuracy = np.mean(accuracy_scores)

print("Accuracy scores for each fold:", accuracy_scores)
print("Average Accuracy:", average_accuracy)


In [ ]:
start_time_single_prediction = time.time()
y_svm = clf.predict(X_test)#svm
single_prediction_time = time.time() - start_time_single_prediction
print("Single Prediction time:", single_prediction_time/len(y_test), "seconds")

print("Accuracy using SVM: {0:.2f}".format(accuracy_score(y_svm,y_test)))#svm

In [ ]:
class_report_svm = classification_report(y_test, y_svm)

print("Classification Report for SVM:\n", class_report_svm)

In [ ]:
from tslearn.svm import TimeSeriesSVC
from sklearn.model_selection import GridSearchCV

# Define the parameter grid including bandwidth values
param_grid = {
    'C': [1,10,100.0],  # values of C to try
    'gamma': [1,10,100.0],  # bandwidth values to try
}

svm_model = TimeSeriesSVC()

grid_search = GridSearchCV(estimator=svm_model, param_grid=param_grid, scoring='accuracy')

grid_search.fit(X_train, y_train)

# Print the best parameters found
print("Best parameters:", grid_search.best_params_)

# Get the best model
best_svm_model = grid_search.best_estimator_

# Make predictions on the test set using the best model
y_pred = best_svm_model.predict(X_test)


In [ ]:
class_labels = ["Default","Stop", "Speed_Up", "Slow_Down", "Change_Direction", "Repeat", "Reset"]
# Define a mapping from numeric labels to class names
label_mapping = {
    0: "Default",
    1: "Stop",
    2: "Speed_Up",
    3: "Slow_Down",
    4: "Change_Direction",
    5: "Repeat",
    6: "Reset"
}

y_test_mapped = [label_mapping[label] for label in y_test]
y_pred_mapped = [label_mapping[label] for label in y_svm]

conf_matrix = confusion_matrix(y_test_mapped, y_pred_mapped, labels=class_labels)

confusion_df = pd.DataFrame(conf_matrix, index=class_labels, columns=class_labels)
confusion_df

In [ ]:
conf_matrix = confusion_matrix(y_test_mapped, y_pred_mapped, labels=class_labels)
confusion_df = pd.DataFrame(conf_matrix, index=class_labels, columns=class_labels)
confusion_df_percent = confusion_df.divide(confusion_df.sum(axis=1), axis=0) * 100

# Generate annotations with both actual number and percentage
annotations = confusion_df.astype(str) + '\n' + confusion_df_percent.round(2).astype(str) + '%'

plt.figure(figsize=(10, 8))
sns.heatmap(confusion_df, annot=annotations, fmt='', cmap="Blues", cbar_kws={'label': 'Counts'})
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')

plt.show()